In [1]:
import requests
import pandas as pd
import time
import random
import asyncio
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
import nest_asyncio
nest_asyncio.apply()

In [2]:
url = 'https://www.rtings.com/tv/reviews/best/tvs-on-the-market'
driver = webdriver.Chrome('C:\Program Files\Python_Drivers\chromedriver.exe')
driver.get(url)

C:\Users\David\AppData\Local\Temp/ipykernel_3956/2295237839.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\Program Files\Python_Drivers\chromedriver.exe')


In [3]:
#Getting all tvs with good reviews
best_tvs = driver.page_source
soup_tv = BeautifulSoup(best_tvs,'html.parser')
winner_tvs = soup_tv.select('div[class = "recommendation_page-body"] > ol > li')
recommended_tvs = soup_tv.select('div[class = "recommendation_page-section"] > div > ul > li ')

In [4]:
def append_details(pLists, winner = False, href = False):
    list_details = []
    if href:
        for link in pLists:
            link_parse = link.split('/')
            link_model = link_parse[-1]
            link_brand = link_parse[-2]
            list_details.append({'model': link_model, 'brand':link_brand, 'url': link, 'winner':winner })
    return list_details

In [5]:
#################################################
# Getting links of winners and recommended ones #
#################################################

#Getting links to visit
# winners
links = ['http://rtings.com'+ y['href'] for x in winner_tvs for y in x.select('div > div > div > a')]
# recommended
links_rec = ['http://rtings.com'+ y['href'] for x in recommended_tvs for y in x.select('div > ul > li > a')]
tv_details = append_details(links, True, True) + append_details(links_rec, False, True) 

tv_details1 = tv_details.copy()

In [6]:
async def get_content(url):
    req = loop.run_in_executor(None, requests.get, url)
    response = await req
    soup = BeautifulSoup(response.text,'html.parser')
    return soup 

In [7]:
time.sleep(random.randint(7,15))
driver.quit()

loop = asyncio.get_event_loop()
for j in range( len(tv_details) ):
    soup = loop.run_until_complete(get_content(tv_details[j]['url']) )
    #getting all clasification rating
    print(tv_details[j]['url'])
    rating_key_list = []
    reviews = {}
    category = ""
    # Getting all overview about each tv
    for overview in soup.select('div[class = "usage_card-details"] > div > div'): #main div
        #getting rating
        for rating in overview.select('div > span'):
            review = rating.text.strip('\n ')
            if review != '':
                rating_key_list.append(review)

        try:
            if len(rating_key_list) > 0:
                category = rating_key_list[1] # getting a category that is used in followings for loops
                reviews[category] = {'Value':rating_key_list[0]}
                rating_key_list.clear()
        except:
            pass
        
        #getting summary per rating
        for summary in overview.select('div > div > div > p'):
            reviews[category]['Opinion'] = summary.text

        for summary in overview.select('div[class="product_page-summary-group"]'):
            pov = summary.text.strip().split('\n')
            pro_or_con = ''
            for pov1 in pov:
                if pov1 != '' and pov1 != '      ':
                    if pov1 == 'Pros':
                        pro_or_con = 'Pros'
                        reviews[category][pro_or_con] = []
                    elif pov1 == 'Cons':
                        pro_or_con = 'Cons'
                        reviews[category][pro_or_con] = []
                    else:
                        reviews[category][pro_or_con].append(pov1)
   
    tv_details1[j]['Overview'] = reviews

    #Getting all inputs
    inputs_list = []
    for inputs in soup.select('div[data-id="7629"] > div span'):
        if inputs.text.strip() != '—':
            inputs_list.append(inputs.text.strip())

    inputs_list = inputs_list[1:]
    tv_details1[j]['Inputs'] = dict(zip(inputs_list[::2], inputs_list[1::2]))

http://rtings.com/tv/reviews/lg/c1-oled
http://rtings.com/tv/reviews/sony/a90j-oled
http://rtings.com/tv/reviews/samsung/qn90a-qled
http://rtings.com/tv/reviews/hisense/u8g
http://rtings.com/tv/reviews/hisense/u6g
http://rtings.com/tv/reviews/sony/a80j-oled
http://rtings.com/tv/reviews/tcl/6-series-r646-2021-qled
http://rtings.com/tv/reviews/vizio/p-series-quantum-x-2020
http://rtings.com/tv/reviews/samsung/qn85a-qled
http://rtings.com/tv/reviews/sony/x85j
http://rtings.com/tv/reviews/vizio/m7-series-quantum-2021
http://rtings.com/tv/reviews/lg/g1-oled
http://rtings.com/tv/reviews/lg/qned90
http://rtings.com/tv/reviews/tcl/5-series-s546-2021-qled
http://rtings.com/tv/reviews/hisense/u7g
http://rtings.com/tv/reviews/sony/x95j
http://rtings.com/tv/reviews/vizio/p-series-quantum-2021
http://rtings.com/tv/reviews/lg/a1-oled
http://rtings.com/tv/reviews/sony/x90j


In [8]:
tv_details1 

[{'model': 'c1-oled',
  'brand': 'lg',
  'url': 'http://rtings.com/tv/reviews/lg/c1-oled',
  'winner': True,
  'Overview': {'Mixed Usage': {'Value': '8.8',
    'Opinion': "The LG C1 OLED is an amazing all-around TV. It has stunning picture quality thanks to its near-infinite contrast ratio, which is great for watching movies or gaming in the dark. The near-instantaneous response time makes motion look exceptionally clear in fast-moving games and sports, and its low input lag is great for gaming or use as a PC monitor. While it doesn't get very bright for HDR, its high contrast helps it deliver an impressive HDR experience.",
    'Pros': ['Near-infinite contrast ratio.',
     'Near-instantaneous response time.',
     'Wide viewing angles.',
     'Perfect blacks with no visible blooming.'],
    'Cons': ['Risk of permanent burn-in.',
     'May not be bright enough for very bright or sunny rooms.']},
   'Movies': {'Value': '9.3',
    'Opinion': "The LG C1 is fantastic for watching movies. 